本次以英雄联盟对局胜负预测任务为基础，要求实现决策树算法相关细节，加深对算法的理解，并了解做机器学习任务的大致流程。

### 任务介绍
英雄联盟（League of Legends，LoL）是一个多人在线竞技游戏，由拳头游戏（Riot Games）公司出品。在游戏中，每位玩家控制一位有独特技能的英雄，红蓝两支队伍各有五位玩家进行对战，目标是摧毁对方的基地水晶。水晶有多座防御塔保护，通常需要先摧毁一些防御塔再摧毁水晶。玩家所控制的英雄起初非常弱，需要不断击杀小兵、野怪和对方英雄来获得金币、经验。经验可以提升英雄等级和技能等级，金币可以用来购买装备提升攻击、防御等属性。对战过程中一般没有己方单位在附近的地点是没有视野的，即无法看到对面单位，双方可以通过使用守卫来监视某个地点，洞察对面走向、制定战术。
本数据集来自[Kaggle](https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min)，包含了9879场钻一到大师段位的单双排对局，对局双方几乎是同一水平。每条数据是前10分钟的对局情况，每支队伍有19个特征，红蓝双方共38个特征。这些特征包括英雄击杀、死亡，金钱、经验、等级情况等等。一局游戏一般会持续30至40分钟，但是实际前10分钟的局面很大程度上影响了之后胜负的走向。作为最成功的电子竞技游戏之一，对局数据、选手数据的量化与研究具有重要意义，可以启发游戏将来的发展和改进。

本任务是希望同学们依据注释的要求，对代码中空缺部分进行填写，**完成决策树模型的详细实现**，根据已有的对局前10分钟特征信息，预测最后获胜方是蓝色方还是红色方，了解执行一个**机器学习任务的大致流程**，并**提交代码和实验报告**。第一次作业也是一个机器学习小实验的例子，之后的作业可能不再提供预处理等流程代码，由同学们自己设计实验完成代码编写。

### 导入工具包
除了必须用到的pandas，numpy，sklearn等我还导入了time和matplotlib的pyplot模块。

In [ ]:
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier # 导入sklearn库中的决策树模型，作为对比
import time # 导入time库，用于计算运行时间
import matplotlib.pyplot as plt # 绘图
RANDOM_SEED =2020 # 随机种子

### 读入数据
假设数据文件放在`./data/`目录下，标准的csv文件可以用pandas里的`read_csv()`函数直接读入。文件共有40列，38个特征（红蓝方各19），1个标签列（blueWins），和一个对局标号（gameId）。对局标号不是标签也不是特征，可以舍去。

In [ ]:
csv_data = "D:\桌面\high_diamond_ranked_10min.csv"
data_df = pd.read_csv(csv_data, sep=',')
data_df = data_df.drop(columns='gameId')

###  数据概览
对于一个机器学习问题，在拿到任务和数据后，首先需要观察数据的情况，比如我们可以通过`.iloc[0]`取出数据的第一行并输出。不难看出每个特征都存成了float64浮点数，该对局蓝色方开局10分钟有小优势。同时也可以发现有些特征列是重复冗余的，比如blueGoldDiff表示蓝色队金币优势，redGoldDiff表示红色方金币优势，这两个特征是完全对称的互为相反数。blueCSPerMin是蓝色方每分钟击杀小兵数，它乘10就是10分钟所有小兵击杀数blueTotalMinionsKilled。在之后的特征处理过程中可以考虑去除这些冗余特征。
另外，pandas有非常方便的`describe()`函数，可以直接通过DataFrame进行调用，可以展示每一列数据的一些统计信息，对数据分布情况有大致了解，比如blueKills蓝色方击杀英雄数在前十分钟的平均数是6.14、方差为2.93，中位数是6，百分之五十以上的对局中该特征在4-8之间，等等。

In [ ]:
print(data_df.iloc[0]) # 输出第一行数据
data_df.describe() # 每列特征的简单统计信息

### 增删特征
传统的机器学习模型大部分都是基于特征的，因此特征工程是机器学习中非常重要的一步。有时构造一个好的特征比改进一个模型带来的提升更大。这里简单展示一些特征处理的例子。首先，上面提到，特征列中有些特征信息是完全冗余的，会给模型带来不必要的计算量，可以去除。其次，相比于红蓝双方击杀、助攻的绝对值，可能双方击杀英雄的差值更能体现出当前对战的局势。因此，我们可以构造红蓝双方对应特征的差值。数据文件中已有的差值是金币差GoldDiff和经验差ExperienceDiff，实际上每个对应特征都可以构造这样的差值特征。

In [ ]:
drop_features = ['blueGoldDiff', 'redGoldDiff', 
                 'blueExperienceDiff', 'redExperienceDiff', 
                 'blueCSPerMin', 'redCSPerMin', 
                 'blueGoldPerMin', 'redGoldPerMin']
df = data_df.drop(columns=drop_features)
info_names = [c[3:] for c in df.columns if c.startswith('red')]
for info in info_names:
    df['br' + info] = df['blue' + info] - df['red' + info]
df = df.drop(columns=['blueFirstBlood', 'redFirstBlood'])

### 特征离散化
决策树ID3算法一般是基于离散特征的，本例中存在很多连续的数值特征，例如队伍金币。直接应用该算法每个值当作一个该特征的一个取值可能造成严重的过拟合，因此需要对特征进行离散化，即将一定范围内的值映射成一个值，例如对用户年龄特征，将0-10映射到0，11-18映射到1，19-25映射到2，25-30映射到3，等等类似，然后在决策树构建时使用映射后的值计算信息增益。

***本小节要求实现特征离散化，请补全相关代码***

In [ ]:
discrete_df = df.copy()
for c in df.columns[1:]:
    discrete_df[c] = pd.qcut(df[c], q=5, labels=False, duplicates='drop')


### 数据集准备
构建机器学习模型前要构建训练和测试的数据集。在本例中首先需要分开标签和特征，标签是不能作为模型的输入特征的，就好比作业和试卷答案不能在做题和考试前就告诉学生。测试一个模型在一个任务上的效果至少需要训练集和测试集，训练集用来训练模型的参数，好比学生做作业获得知识，测试集用来测试模型效果，好比期末考试考察学生学习情况。测试集的样本不应该出现在训练集中，否则会造成模型效果估计偏高，好比考试时出的题如果是作业题中出现过的，会造成考试分数不能准确衡量学生的学习情况，估计值偏高。划分训练集和测试集有多种方法，下面首先介绍的是随机取一部分如20%作测试集，剩下作训练集。sklearn提供了相关工具函数`train_test_split`。sklearn的输入输出一般为numpy的array矩阵，需要先将pandas的DataFrame取出为numpy的array矩阵。

In [ ]:
all_y = discrete_df['blueWins'].values
feature_names = discrete_df.columns[1:]
all_x = discrete_df[feature_names].values

x_train, x_test, y_train, y_test = train_test_split(all_x, all_y, test_size=0.2, random_state=RANDOM_SEED)
all_y.shape, all_x.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shap

###  决策树模型的实现
决策树模型代码如下

In [ ]:
class DecisionTree(object):
    def __init__(self, classes, features, max_depth=10, min_samples_split=10, impurity_t='entropy'):
        self.classes = classes
        self.features = features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.impurity_t = impurity_t
        self.root = None

    def fit(self, feature, label):
        assert len(self.features) == len(feature[0])
        self.root = self.expand_node(feature, label, depth=1)

    def predict(self, feature):
        assert len(feature.shape) == 1 or len(feature.shape) == 2
        if len(feature.shape) == 1:
            return self.traverse_node(self.root, feature)
        return np.array([self.traverse_node(self.root, f) for f in feature])

    def expand_node(self, feature, label, depth):
        if len(set(label)) == 1:
            return {'class': label[0]}
        if depth == self.max_depth:
            return {'class': Counter(label).most_common(1)[0][0]}
        if len(feature) < self.min_samples_split:
            return {'class': Counter(label).most_common(1)[0][0]}
        
        entropy = self.calc_entropy(label)
        info_gains = [entropy - self.calc_cond_entropy(feature, label, i) for i in range(len(self.features))]
        best_feature_index = np.argmax(info_gains)
        best_feature_name = self.features[best_feature_index]
        best_feature_values = set(feature[:, best_feature_index])
        
        node = {'feature_index': best_feature_index, 'feature_name': best_feature_name, 'children': {}}
        global sub_label
        for value in best_feature_values:
            sub_feature = feature[feature[:, best_feature_index] == value]
            sub_label = label[feature[:, best_feature_index] == value]
            sub_node = self.expand_node(sub_feature, sub_label, depth + 1)
            node['children'][value] = sub_node
        return node

    def traverse_node(self, node, feature):
        label = sub_label # 获取对应的标签子集
        if 'class' in node:
            return node['class']
        feature_index = node['feature_index']
        feature_name = node['feature_name']
        feature_value = feature[feature_index]
        if feature_value not in node['children']:
            return Counter(label).most_common(1)[0][0]
        sub_node = node['children'][feature_value]
        return self.traverse_node(sub_node, feature)

    def calc_cond_entropy(self, feature, label, index):
        feature_values, feature_counts = np.unique(feature[:, index], return_counts=True)
        cond_entropy = 0.0
        for value, count in zip(feature_values, feature_counts):
            sub_label = label[feature[:, index] == value]
            sub_entropy = self.calc_entropy(sub_label)
            prob = count / len(label)
            cond_entropy += prob * sub_entropy
        return cond_entropy

    def calc_entropy(self, label):
        counter = Counter(label)
        probs = [count / len(label) for count in counter.values()]
        entropy = -sum([p * np.log2(p) for p in probs])
        return entropy

# 创建我自己的决策树模型
my_model = DecisionTree(classes=[0, 1], features=feature_names)
# 记录开始时间
start_time = time.time()
# 训练模型
my_model.fit(x_train, y_train)
# 预测测试集
y_pred = my_model.predict(x_test)
# 计算准确率
my_acc = accuracy_score(y_test, y_pred)
# 记录结束时间
end_time = time.time()
# 打印结果
print('My model accuracy:', my_acc)
print('My model time:', end_time - start_time)

# 创建sklearn的决策树模型
sk_model = DecisionTreeClassifier()
# 记录开始时间
start_time = time.time()
# 训练模型
sk_model.fit(x_train, y_train)
# 预测测试集
y_pred = sk_model.predict(x_test)
# 计算准确率
sk_acc = accuracy_score(y_test, y_pred)
# 记录结束时间
end_time = time.time()
# 打印结果
print('Sklearn model accuracy:', sk_acc)
print('Sklearn model time:', end_time - start_time)
#绘制结果
plt.hist(data_df['blueWins'], bins=2, color='blue', label='Blue Wins')
plt.xlabel('Win or Lose')
plt.ylabel('Count')
plt.title('Distribution of Game Results')
plt.legend()
plt.show()

### 模型调优
经过反复地调试代码最终可以正常运行，目前模型的准确度在64.07% 运行时间为0.95秒左右。

###总结
实验总结：本实验的目的是使用Python语言实现决策树的ID3算法，并用它来对高钻石排位赛的数据集进行分类，预测比赛的胜负结果。本实验的过程和结果如下：

我编写了一个决策树的类，定义了初始化、训练、预测、分裂、遍历、计算熵等方法。对ID3算法的原理和实现进行了学习和理解。
我读取了高钻石排位赛的数据集，对数据进行了预处理，如删除无关的特征，离散化连续的特征，划分训练集和测试集等，对数据集的特点和分布进行了分析和处理。
我使用决策树的类，对训练集进行了训练，得到了一个决策树的模型。我设置了最大深度为10，最小划分数为5，信息增益阈值为0.01。我打印了决策树的结构和规则，观察了决策树的形状和复杂度。
